# FastAPI:
- It is a Python web framework optimized for building APIs.
- It uses Python [type hints](https://realpython.com/python-type-checking/) and has built-in support for [async operations](https://realpython.com/async-io-python/).
- It's built on top of [Starlette](https://www.starlette.io/) and [Pydantic](https://pydantic-docs.helpmanual.io/) and is very performant.
- Below is an example:

```python
# app.py
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()

def _find_next_id():
    return max(country.country_id for country in countries) + 1

class Country(BaseModel):
    # Pydantic formats!
    country_id: int = Field(default_factory=_find_next_id, alias="id")
    name: str
    capital: str
    area: int

countries = [
    Country(id=1, name="Thailand", capital="Bangkok", area=513120),
    Country(id=2, name="Australia", capital="Canberra", area=7617930),
    Country(id=3, name="Egypt", capital="Cairo", area=1010408),
]

@app.get("/countries")
async def get_countries():
    return countries

@app.post("/countries", status_code=201)
async def add_country(country: Country):
    countries.append(country)
    return country
```
- This application uses the features of FastAPI to build a REST API for the same ``country`` data we’ve seen in the other examples.


## Installation:
- To install FastAPI, with ``pip``:
```sh
python -m pip install fastapi
```
- We’ll also need to install uvicorn\[standard\], a server that can run FastAPI applications:
```sh
python -m pip install uvicorn[standard]
```
- To run, save the code in a file called ``app.py`` and run the below:
```sh
> uvicorn app:app --reload
INFO: Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```
- When we open up a browser and go to http://127.0.0.1:8000/countries. We’ll see FastAPI respond with this:

```json
[
    {
        "id": 1,
        "name":"Thailand",
        "capital":"Bangkok",
        "area":513120
    },
    {
        "id": 2,
        "name":"Australia",
        "capital":"Canberra",
        "area":7617930
    },
    {
        "id": 3,
        "name":"Egypt",
        "capital":"Cairo",
        "area":1010408
    }
]
```
- We can also add a new ``country`` by sending a ``POST`` request to ``/countries``:
```sh
curl -i http://127.0.0.1:8000/countries \
-X POST \
-H 'Content-Type: application/json' \
-d '{"name":"Germany", "capital": "Berlin", "area": 357022}' \
-w '\n'
```
- Response:
```json
HTTP/1.1 201 Created
content-type: application/json
...

{"id":4,"name":"Germany","capital":"Berlin","area": 357022}
```

- Confirm this with ``GET /countries``:
```sh
curl -i http://127.0.0.1:8000/countries -w '\n'
```
- FastAPI returns a JSON list including the new country we just added:
```json
HTTP/1.1 200 OK
content-type: application/json
...

[
    {
        "id":1,
        "name":"Thailand",
        "capital":"Bangkok",
        "area":513120,
    },
    {
        "id":2,
        "name":"Australia",
        "capital":"Canberra",
        "area":7617930
    },
    {
        "id":3,
        "name":"Egypt",
        "capital":"Cairo",
        "area":1010408
    },
    {
        "id":4,
        "name": "Germany",
        "capital": "Berlin",
        "area": 357022
    }
]
```
## Code Overview:
- Like Flask, FastAPI has a focused feature set. 
- It doesn’t try to handle all aspects of web application development. It’s designed to build APIs with modern Python features.
- The subclass ``Country`` describes the structure of the data in the REST API and extends the ``BaseModel`` class:
```python
class Country(BaseModel):
    country_id: int = Field(default_factory=_find_next_id, alias="id")
    name: str
    capital: str
    area: int
```
- *Pydantic* models use Python type annotations to **enforce the data type** for each field in the class. 
- This allows FastAPI to automatically generate JSON, with the correct data types, for API endpoints and also validate incoming JSON.
- If we observe the first line:
```python
country_id: int = Field(default_factory=_find_next_id, alias="id")
```
- ``country_id`` stores an integer for the ID of the ``Country``. It uses the ``Field`` function from *Pydantic* to modify the behavior of ``country_id``. In this example, we’re passing ``Field`` the keyword arguments ``default_factory`` and ``alias``.
  - ``default_factory``: set to \_find_next_id() - specifies a function to run whenever a new ``Country`` is created - return value assigned to ``country_id``.
  - ``alias``: set to ``id`` -- tells FastAPI to output the key ``"id"`` instead of ``"country_id"`` in the JSON -  also means we can use ``id`` when creating a new ``Country``.
```python
countries = [
    Country(id=1, name="Thailand", capital="Bangkok", area=513120),
    Country(id=2, name="Australia", capital="Canberra", area=7617930),
    Country(id=3, name="Egypt", capital="Cairo", area=1010408),
]
```
- We also have two API functions defined in this application:

### 1. GET (get_countries()):
```python
@app.get("/countries")
async def get_countries():
    return countries
```
- FastAPI automatically creates JSON based on the fields in the Pydantic model and sets the right JSON data type from the Python type hints.

### 2. POST (add_country()):
- The Pydantic model also provides a benefit when we make a ``POST`` request to ``/countries``. We can see in the second API function below that the parameter ``country`` has a ``Country`` annotation:
```python
@app.post("/countries", status_code=201)
async def add_country(country: Country):
    countries.append(country)
    return country
```
- This type annotation tells FastAPI to validate the incoming JSON against ``Country``.
- If it doesn’t match, then FastAPI will return an error:

```sh
curl -i http://127.0.0.1:8000/countries \
-X POST \
-H 'Content-Type: application/json' \
-d '{"name":"Germany", "capital": "Berlin"}' \
-w '\n'
```
- The JSON in this request was missing a value for ``area``, so FastAPI should return a response with the status code ``422 Unprocessable Entit``y as well as details about the error. This validation is made possible by the *Pydantic* model.

```json
HTTP/1.1 422 Unprocessable Entity
content-type: application/json
...

{
    "detail": [
        {
            "loc":["body","area"],
            "msg":"field required",
            "type":"value_error.missing"
        }
    ]
}
```

--------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------